# Finding Maximum Flow Using Linear Programming
We've already seen how we can use the Ford-Fulkerson algorithm to find the maximum flow from St. Petersburg to Moscow.
However, it turns out we can also use linear programming to find the maximum cut of a network flow!
We do that here.<br>
To see a picture of the network we are working with, see the file titled "network.pdf".

In [3]:
#import necessary packages
import numpy as np
from scipy.optimize import linprog

First create a vector with the labels of the edges. <br>
This is mainly for bookkeeping: it helps clarify where constraints ought to go and gives context to the final solution.

In [4]:
labs = ["e_sA", "e_sB", "e_AC", "e_AD", "e_BC", "e_BE", "e_CE", "e_CG", "e_DI", "e_EF", "e_EG", "e_FH", "e_GH", "e_Ht", "e_GI", "e_IJ", "e_Jt"]
nums = np.arange(17)

Now, create our objective function. Since we are maximizing flow, we want to maximize the sum of all our edges that lead into our target.<br>
I've labeled my edges in such a way so that our last and fourth from last edges are leading into the target. <br>
linprog only solves minimization problems, so we multiply our objective function by -1. <br>

In [5]:
obj = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1]
#check length of vector since there are so many edges
#len(obj)

Create constraints that tell us the capacity of each edge

In [6]:
lhs_capacity = np.eye(17)
rhs_capacity = [3, 3, 4, 2, 2, 2, 3, 4, 5, 3, 4, 4, 3, 5, 3, 3, 1]
#len(rhs_capacity)

It's nice for visualization to have a matrix with each edge's index, label, and capacity.

In [7]:
labeled_capacity = np.column_stack([nums, labs, rhs_capacity])
print(labeled_capacity)

[['0' 'e_sA' '3']
 ['1' 'e_sB' '3']
 ['2' 'e_AC' '4']
 ['3' 'e_AD' '2']
 ['4' 'e_BC' '2']
 ['5' 'e_BE' '2']
 ['6' 'e_CE' '3']
 ['7' 'e_CG' '4']
 ['8' 'e_DI' '5']
 ['9' 'e_EF' '3']
 ['10' 'e_EG' '4']
 ['11' 'e_FH' '4']
 ['12' 'e_GH' '3']
 ['13' 'e_Ht' '5']
 ['14' 'e_GI' '3']
 ['15' 'e_IJ' '3']
 ['16' 'e_Jt' '1']]


Now, create constraints that make sure flow into each node equals flow out of each node. <br>
* For edges leading out of source, we have that flow_in-flow_out = e_Jt + e_Gt. <br>
  * This is bad! We want a numerical value on the RHS. Rewrite as flow_in-flow_out - e_Jt - e_Ht = 0. <br>
* For edges leading out of target, we have that flow_in-flow_out = -(e_Jt + e_Gt). <br>
  * Again, this is bad! We want a numerical value on the RHS. Rewrite as flow_in-flow_out + e_Jt + e_Ht = 0. 

In [8]:
lhs_balance = [[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1], #flow leaving start
               [-1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #flow in/out of A
               [0, -1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #flow in/out of B
               [0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #flow in/out of C
               [0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], #flow in/out of D
               [0, 0, 0, 0, 0, -1, -1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], #flow in/out of E
               [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0], #flow in/out of F
               [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, 0, 1, 0, 0], #flow in/out of G
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0], #flow in/out of H
               [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 1, 0], #flow in/out of I
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1], #flow in/out of J
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] #flow in target

rhs_balance = np.zeros(12) #flow out of each note must equal flow into eahc node (see above for explanation for source/target).

In [9]:
#Create bounds for each variable
bnd = [(0, float("inf")), #bounds of s
       (0, float("inf")), #bounds of A
       (0, float("inf")), #bounds of B
       (0, float("inf")), #bounds of C
       (0, float("inf")), #bounds of D
       (0, float("inf")), #bounds of E
       (0, float("inf")), #bounds of F
       (0, float("inf")), #bounds of G
       (0, float("inf")), #bounds of H
       (0, float("inf")), #bounds of I
       (0, float("inf")), #bounds of J
       (0, float("inf"))] #bounds of t

Use linprog to find our optimal solution. <br>
* c: objective function <br>
* A_ub: LHS of constraints that have inequalities. Should be matrix w/ nrows=number of constraints, ncols=number of variables. <br>
* b_ub: RHS of constraints that have inequalities. Should be vector w/ as many entries as there are constraints. <br>
* A_eq: LHS of constraints that have equalities. Should be matrix w/ nrows=number of constraints, ncols=number of variables. <br>
* b_eq: RHS of constraints that have equalities. Should be vector w/ as many entries as there are constraints. <br>

In [10]:
opt = linprog(c=obj, A_ub=lhs_capacity, b_ub=rhs_capacity, A_eq=lhs_balance, b_eq=rhs_balance, method="revised simplex")
opt

/var/folders/3h/6frnxss502n097jxqmmf66l00000gn/T/ipykernel_52298/4143925119.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  opt = linprog(c=obj, A_ub=lhs_capacity, b_ub=rhs_capacity, A_eq=lhs_balance, b_eq=rhs_balance, method="revised simplex")


     con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
     fun: -6.0
 message: 'Optimization terminated successfully.'
     nit: 15
   slack: array([0., 0., 2., 1., 1., 0., 2., 2., 4., 0., 4., 1., 1., 0., 3., 2., 0.])
  status: 0
 success: True
       x: array([3., 3., 2., 1., 1., 2., 1., 2., 1., 3., 0., 3., 2., 5., 0., 1., 1.])

This not only tells us our optimal solution, but we also get some extra helpful and fun information.

* fun: gives us the optimal solution. Here, we see that the maximum flow is 6 (recall that we had to turn our "maximize" into a "minimize"). <br>
* nit: gives us the number of iterations used to solve this problem... Here, we needed 15. <br>
* x: tells us the flow on each edge that will produce our maximum flow. 

Now, let's use the output 'x' above to find the paths and weights that obtain the maximal flow.

In [23]:
x = opt['x']
# len(x)
optimal_flow = np.column_stack([labs, x])
print(optimal_flow)

[['e_sA' '3.0']
 ['e_sB' '3.0']
 ['e_AC' '2.0']
 ['e_AD' '1.0']
 ['e_BC' '1.0']
 ['e_BE' '2.0']
 ['e_CE' '1.0']
 ['e_CG' '2.0']
 ['e_DI' '1.0']
 ['e_EF' '3.0']
 ['e_EG' '0.0']
 ['e_FH' '3.0']
 ['e_GH' '2.0']
 ['e_Ht' '5.0']
 ['e_GI' '0.0']
 ['e_IJ' '1.0']
 ['e_Jt' '1.0']]
